# NTK classifier for Cora

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np
from exp_ntk_certify import run

In [4]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [5]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        n_per_class = 20,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

certificate_params = dict(
    n_adversarial = 10, # number adversarial nodes
    perturbation_model = "l0",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [6]:
def run_exp(n_seeds, data_params, model_params, certificate_params,
            verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, certificate_params,
                  verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [10]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "krr"
model_params["cache_size"] = 10000
data_params["dataset"] = "cora_ml"
other_params["device"] = 0
certificate_params["n_adversarial"] = 10
certificate_params["delta"] = 0.01
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-11 18:30:25 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-01-11 18:30:25 (INFO): data_params: {'dataset': 'cora_ml', 'learning_setting': 'inductive', 'specification': {'n_per_class': 20, 'fraction_test': 0.1, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True, 'seed': 0}}
2024-01-11 18:30:25 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'krr', 'cache_size': 10000}
2024-01-11 18:30:25 (INFO): certification_params: {'n_adversarial': 10, 'perturbation_model': 'l0', 'delta': 0.01}
2024-01-11 18:30:25 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-11 18:30:25 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-11 18:30:25 (INFO): seed: 0
2024-01-11 18:30:25 (INFO): Currently on gpu device cuda:0
2024-01-11 18:30:26 (INFO): X.min(): 0.0
2024-01-11 18:30:26 (INFO): X.max(): 1.0
2024-01-

{'accuracy': 0.813380241394043,
 'accuracy_ub': 0.31338027119636536,
 'accuracy_lb': 0.7077464461326599,
 'accuracy_cert': 0.47887322306632996,
 'accuracy_cert_unrobust': 0.32042253017425537,
 'min_ypred': -0.679108707845861,
 'max_ypred': 1.9279342579468663,
 'min_ylb': -0.7205013733062592,
 'max_ylb': 1.9658152324249596,
 'min_yub': -1.5159283394753253,
 'max_yub': 8.061858523859527,
 'min_ntklb': 0.4616764139530176,
 'max_ntklb': 139.69693436430674,
 'min_ntkub': 2.3331038349874444,
 'max_ntkub': 269.95186495266415,
 'min_ntklabeled': 3.518943905429874,
 'max_ntklabeled': 142.08693269172264,
 'min_ntkunlabeled': 2.644019167238364,
 'max_ntkunlabeled': 47.537976680322174,
 'cond': 26316.41952343723}

In [8]:
y_pred = torch.tensor(([1, 2, 3], [1, 3, 2], [3, 2, 1]))
y_lb = torch.tensor(([0, 1, 2], [0, 2, 1], [2, 1, 0]))
y_ub = torch.tensor(([3, 3, 4], [1, 3, 1.5], [3, 4, 1]))
n = y_pred.shape[0]
pred_orig = y_pred.argmax(1)
print(pred_orig)
pred_orig_lb = y_lb[range(n), pred_orig]
print(pred_orig_lb)
mask = torch.ones(y_ub.shape, dtype=torch.bool).to(y_ub.device)
mask[range(n), pred_orig] = False
pred_other_ub = y_ub[mask].reshape((n, y_ub.shape[1]-1))
print(pred_other_ub)
count_beaten = (pred_other_ub > pred_orig_lb.reshape(-1, 1)).sum(dim=1)
print(count_beaten)
print(((count_beaten == 0).sum() / n).cpu().item())


tensor([2, 1, 0])
tensor([2, 2, 2])
tensor([[3.0000, 3.0000],
        [1.0000, 1.5000],
        [4.0000, 1.0000]])
tensor([2, 0, 1])
0.3333333432674408


# SVM

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 1
data_params["dataset"] = "cora"
model_params["pred_method"] = "krr"
n_seeds = 2
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)